# 🎯 Advanced Grasp Detection v7

## New in v7
| Feature | Description |
|---------|-------------|
| **📏 Camera Height Calibration** | ระบุความสูงกล้องจากพื้นเพื่อ depth แม่นขึ้น |
| **🔬 Multi-Object Depth Calibration** | Calibrate ด้วยวัตถุหลายชิ้น |
| **📐 Frame Grid + Object Grid** | Grid ทั้งเฟรม + Grid บนวัตถุแต่ละชิ้น |

## 1️⃣ Imports

In [1]:
import sys
import cv2
import numpy as np
import time
import socket
import serial
import torch
from collections import deque
from ultralytics import YOLO

sys.path.append('Depth-Anything-V2')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print("✓ Imports loaded")

PyTorch: 2.9.1+cpu
CUDA: False
✓ Imports loaded


## 2️⃣ Hardware Configuration

In [2]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.array([
    [0.005703976266962427, -0.3265299161278153, 88.58634169557483],
    [-0.47704058225560797, 0.015355046930804153, 172.0941543570439],
    [-0.00029949919510557677, 0.00018728182448344945, 1.0],
], dtype=np.float32)

print("✓ Hardware config")

✓ Hardware config


---
# 🔧 CALIBRATION SECTION
---

## 📐 Calibration 1: PIXELS_PER_MM

In [5]:
# Uncomment เพื่อ calibrate
# ลากเส้นบนไม้บรรทัด → ENTER → ใส่ขนาดจริง (mm)

drawing = False
start_pt, end_pt = None, None
px_dist = 0

def cb(event, x, y, flags, param):
    global drawing, start_pt, end_pt, px_dist
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing, start_pt = True, (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        end_pt = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing, end_pt = False, (x, y)
        px_dist = np.sqrt((end_pt[0]-start_pt[0])**2 + (end_pt[1]-start_pt[1])**2)
        print(f"Distance: {px_dist:.1f} px")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pixel Cal')
cv2.setMouseCallback('Pixel Cal', cb)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    if start_pt and end_pt: cv2.line(frame, start_pt, end_pt, (0,255,0), 2)
    cv2.imshow('Pixel Cal', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == 13 and px_dist > 0:
        mm = float(input("Real size (mm): "))
        print(f"PIXELS_PER_MM = {px_dist/mm:.4f}")
cap.release()
cv2.destroyAllWindows()


Distance: 201.0 px
Distance: 200.0 px
PIXELS_PER_MM = 2.2222


## 📏 Calibration 2: Camera Height & Depth Scale

**ขั้นตอน:**
1. วัดความสูงกล้องจากพื้น (mm) → ใส่ `CAMERA_HEIGHT_MM`
2. เตรียมวัตถุ 2-3 ชิ้นที่รู้ความสูง
3. Run cell → กด C calibrate พื้น
4. วางวัตถุ → คลิก → ใส่ความสูงจริง
5. ทำซ้ำกับวัตถุอื่น → ได้ค่า DEPTH_Z_SCALE เฉลี่ย

In [6]:
# === ADVANCED DEPTH CALIBRATION ===
# Run หลังจาก Load Depth Model (Section 4)

# ⚠️ วัดความสูงกล้องจากพื้น (mm)
CAMERA_HEIGHT_MM = 630  # 🔴 แก้ค่านี้!

print(f"Camera height: {CAMERA_HEIGHT_MM}mm")
print("Run depth calibration after loading models...")

Camera height: 630mm
Run depth calibration after loading models...


## 🦾 Calibration 3: Gripper Test

---
## 3️⃣ Configuration
---

In [28]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.222
DEPTH_Z_SCALE = 26.1660
CAMERA_HEIGHT_MM = 630  # ความสูงกล้องจากพื้น

# === Z Heights ===
Z_FLOOR = -64
Z_SAFE = -40
Z_APPROACH = -55

# === Drop Position ===
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# === Gripper (NO MARGIN) ===
GRIPPER_SERVO_OPEN_ANGLE = 22
GRIPPER_SERVO_CLOSE_ANGLE = 96
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_MIN_WIDTH_MM = 0
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 0

# === Detection ===
MIN_OBJECT_AREA = 1000
YOLO_CONFIDENCE = 0.25

# === Depth Model ===
DEPTH_MODEL_PATH = 'Depth-Anything-V2/checkpoints/depth_anything_v2_vits.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# === Grid Display ===
SHOW_FRAME_GRID = True
SHOW_OBJECT_GRID = True
FRAME_GRID_SIZE_MM = 20
OBJECT_GRID_SIZE_MM = 5

print("✓ Configuration loaded")
print(f"  Camera height: {CAMERA_HEIGHT_MM}mm")
print(f"  Frame grid: {FRAME_GRID_SIZE_MM}mm, Object grid: {OBJECT_GRID_SIZE_MM}mm")

✓ Configuration loaded
  Camera height: 630mm
  Frame grid: 20mm, Object grid: 5mm


## 4️⃣ Load Models

In [29]:
from depth_anything_v2.dpt import DepthAnythingV2

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
}

print("Loading DepthAnything V2...")
depth_model = DepthAnythingV2(**model_configs['vits'])
depth_model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location='cpu'))
depth_model = depth_model.to(DEVICE).eval()
print(f"✅ Depth model on {DEVICE}")

print("Loading YOLOv8...")
yolo_model = YOLO('yolov8n.pt')
print("✅ YOLO loaded")

Loading DepthAnything V2...
✅ Depth model on cpu
Loading YOLOv8...
✅ YOLO loaded


## 📏 Run Depth Calibration (Multi-Object)

In [20]:
robot.home()

🤖 HOME...


In [21]:
# === MULTI-OBJECT DEPTH CALIBRATION ===
print("="*60)
print("📏 DEPTH CALIBRATION (Multi-Object)")
print("="*60)
print("1. กด C = Calibrate พื้น (เอาวัตถุออก)")
print("2. วางวัตถุที่รู้ความสูง → คลิก → ใส่ความสูง (mm)")
print("3. ทำซ้ำกับวัตถุอื่นเพื่อความแม่นยำ")
print("4. กด S = Show average scale")
print("5. กด Q = Quit")
print("="*60)

calibration_samples = []
floor_depth = None
click_x, click_y = None, None

def depth_cal_callback(event, x, y, flags, param):
    global click_x, click_y
    if event == cv2.EVENT_LBUTTONDOWN:
        click_x, click_y = x, y

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Depth Calibration')
cv2.setMouseCallback('Depth Calibration', depth_cal_callback)

current_depth_map = None
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame_count += 1
    if frame_count % 5 == 0:
        current_depth_map = depth_model.infer_image(frame)
    
    # Display
    floor_str = f"Floor: {floor_depth:.4f}" if floor_depth else "Floor: NOT SET"
    samples_str = f"Samples: {len(calibration_samples)}"
    cv2.putText(frame, f"{floor_str} | {samples_str} | C=Floor S=Show Q=Quit",
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    
    if calibration_samples:
        avg_scale = np.mean([s['scale'] for s in calibration_samples])
        cv2.putText(frame, f"Avg DEPTH_Z_SCALE = {avg_scale:.2f}",
                   (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    
    cv2.imshow('Depth Calibration', frame)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    
    elif key == ord('c'):
        if current_depth_map is not None:
            h, w = current_depth_map.shape
            center = current_depth_map[h//3:2*h//3, w//3:2*w//3]
            floor_depth = np.median(center)
            print(f"✅ Floor depth: {floor_depth:.4f}")
    
    elif key == ord('s'):
        if calibration_samples:
            print("\n" + "="*60)
            print("📊 Calibration Results:")
            for i, s in enumerate(calibration_samples):
                print(f"   {i+1}. Height={s['height_mm']}mm → Scale={s['scale']:.2f}")
            avg = np.mean([s['scale'] for s in calibration_samples])
            std = np.std([s['scale'] for s in calibration_samples])
            print("="*60)
            print(f"✅ DEPTH_Z_SCALE = {avg:.4f} (±{std:.2f})")
            print("="*60)
    
    # Handle click
    if click_x is not None and current_depth_map is not None and floor_depth is not None:
        # Get depth at click point (small region)
        x, y = click_x, click_y
        h, w = current_depth_map.shape
        x = max(10, min(w-10, x))
        y = max(10, min(h-10, y))
        
        region = current_depth_map[y-10:y+10, x-10:x+10]
        obj_depth = np.median(region)
        raw_diff = obj_depth - floor_depth
        
        if raw_diff > 0:
            try:
                height_mm = float(input(f"\n📏 Object at ({x},{y}) - Enter height (mm): "))
                scale = height_mm / raw_diff
                calibration_samples.append({
                    'height_mm': height_mm,
                    'raw_diff': raw_diff,
                    'scale': scale
                })
                print(f"   ✅ Sample {len(calibration_samples)}: Scale={scale:.2f}")
            except:
                print("   ❌ Invalid input")
        else:
            print(f"   ❌ Depth diff={raw_diff:.4f} (should be > 0)")
        
        click_x, click_y = None, None

cap.release()
cv2.destroyAllWindows()

if calibration_samples:
    final_scale = np.mean([s['scale'] for s in calibration_samples])
    print(f"\n📋 Copy this: DEPTH_Z_SCALE = {final_scale:.4f}")

📏 DEPTH CALIBRATION (Multi-Object)
1. กด C = Calibrate พื้น (เอาวัตถุออก)
2. วางวัตถุที่รู้ความสูง → คลิก → ใส่ความสูง (mm)
3. ทำซ้ำกับวัตถุอื่นเพื่อความแม่นยำ
4. กด S = Show average scale
5. กด Q = Quit
✅ Floor depth: 3.1107
   ✅ Sample 1: Scale=45.27
   ✅ Sample 2: Scale=45.05
   ✅ Sample 3: Scale=8.71
   ✅ Sample 4: Scale=19.05

📊 Calibration Results:
   1. Height=107.0mm → Scale=45.27
   2. Height=107.0mm → Scale=45.05
   3. Height=11.0mm → Scale=8.71
   4. Height=27.0mm → Scale=19.05
✅ DEPTH_Z_SCALE = 29.5190 (±16.06)
   ✅ Sample 5: Scale=12.75

📊 Calibration Results:
   1. Height=107.0mm → Scale=45.27
   2. Height=107.0mm → Scale=45.05
   3. Height=11.0mm → Scale=8.71
   4. Height=27.0mm → Scale=19.05
   5. Height=27.0mm → Scale=12.75
✅ DEPTH_Z_SCALE = 26.1660 (±15.86)
   ❌ Invalid input

📋 Copy this: DEPTH_Z_SCALE = 26.1660


## 5️⃣ Classes Definition

In [22]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            print(f"✅ Gripper on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: self.serial.close()
    
    def send_command(self, cmd):
        if self.serial:
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {width_mm:.1f}mm → {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {width_mm:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None


class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]

print("✓ Gripper & Robot classes")

✓ Gripper & Robot classes


In [23]:
class PreciseSizeDetector:
    """YOLO + Contour + Grid (Frame + Object)"""
    
    def __init__(self, yolo_model, pixels_per_mm):
        self.yolo = yolo_model
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt += np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'rect_size': rect[1],
                        'rect_angle': rect[2],
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt)
                    })
        
        if not objects:
            objects = self.edge_detect(frame)
        
        return objects
    
    def edge_detect(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(cv2.GaussianBlur(gray, (5,5), 0), 50, 150)
        edges = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > MIN_OBJECT_AREA:
                hull = cv2.convexHull(cnt)
                rect = cv2.minAreaRect(hull)
                x,y,w,h = cv2.boundingRect(hull)
                objects.append({
                    'bbox': (x,y,w,h),
                    'center': (x+w//2, y+h//2),
                    'rect': rect,
                    'rect_size': rect[1],
                    'rect_angle': rect[2],
                    'contour': hull,
                    'area': area
                })
        return sorted(objects, key=lambda o: o['area'], reverse=True)
    
    def draw_frame_grid(self, frame):
        """Draw grid on entire frame"""
        if not SHOW_FRAME_GRID: return frame
        h, w = frame.shape[:2]
        grid_px = int(FRAME_GRID_SIZE_MM * self.ppm)
        for x in range(0, w, grid_px):
            cv2.line(frame, (x,0), (x,h), (50,50,50), 1)
        for y in range(0, h, grid_px):
            cv2.line(frame, (0,y), (w,y), (50,50,50), 1)
        cv2.putText(frame, f"Frame Grid: {FRAME_GRID_SIZE_MM}mm", (10, h-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,100,100), 1)
        return frame
    
    def draw_object_grid(self, frame, obj):
        """Draw grid overlay on object"""
        if not SHOW_OBJECT_GRID: return
        
        x, y, w, h = obj['bbox']
        grid_px = int(OBJECT_GRID_SIZE_MM * self.ppm)
        if grid_px < 3: grid_px = 3
        
        # Draw grid inside bbox
        for gx in range(x, x+w, grid_px):
            cv2.line(frame, (gx, y), (gx, y+h), (100,100,255), 1)
        for gy in range(y, y+h, grid_px):
            cv2.line(frame, (x, gy), (x+w, gy), (100,100,255), 1)
        
        # Size labels
        w_mm = w / self.ppm
        h_mm = h / self.ppm
        cv2.putText(frame, f"{w_mm:.0f}mm", (x + w//2 - 15, y - 3),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.35, (100,100,255), 1)
        cv2.putText(frame, f"{h_mm:.0f}mm", (x + w + 3, y + h//2),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.35, (100,100,255), 1)

print("✓ Detector class")

✓ Detector class


In [24]:
class SmartGraspSelector:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        w, h = obj['rect_size']
        angle = obj['rect_angle']
        cx, cy = obj['center']
        aspect = max(w,h) / (min(w,h) + 0.001)
        is_ring = self.detect_ring(obj)
        
        grasps = []
        
        if aspect > 2.0:
            if w < h:
                grip_w, grip_a = w / self.ppm, angle + 90
            else:
                grip_w, grip_a = h / self.ppm, angle
            grasps.append({'center': (cx,cy), 'width_mm': grip_w, 'angle': self.norm(grip_a), 'score': 1.0, 'type': 'narrow'})
            
            alt_w = max(w,h) / self.ppm
            if alt_w <= GRIPPER_MAX_WIDTH_MM:
                grasps.append({'center': (cx,cy), 'width_mm': alt_w, 'angle': self.norm(grip_a+90), 'score': 0.5, 'type': 'alt'})
        
        elif is_ring:
            inner_w = max(w,h) / self.ppm * 0.3
            for i in range(4):
                grasps.append({'center': (cx,cy), 'width_mm': inner_w, 'angle': i*45, 'score': 1.0 if i==0 else 0.8, 'type': 'ring'})
        
        else:
            if w < h:
                grip_w, grip_a = w / self.ppm, angle + 90
            else:
                grip_w, grip_a = h / self.ppm, angle
            grasps.append({'center': (cx,cy), 'width_mm': grip_w, 'angle': self.norm(grip_a), 'score': 1.0, 'type': 'default'})
            
            perp_w = max(w,h) / self.ppm
            if perp_w <= GRIPPER_MAX_WIDTH_MM:
                grasps.append({'center': (cx,cy), 'width_mm': perp_w, 'angle': self.norm(grip_a+90), 'score': 0.6, 'type': 'perp'})
        
        return grasps
    
    def detect_ring(self, obj):
        cnt = obj.get('contour')
        if cnt is None: return False
        hull = cv2.convexHull(cnt)
        return (cv2.contourArea(cnt) / (cv2.contourArea(hull) + 0.001)) < 0.7
    
    def norm(self, a):
        while a > 90: a -= 180
        while a < -90: a += 180
        return a

print("✓ Grasp selector")

✓ Grasp selector


In [25]:
class RobustDepthEstimator:
    """Multi-sample + temporal + camera height aware"""
    
    def __init__(self, model, device='cpu', camera_height_mm=450, history_size=5):
        self.model = model
        self.device = device
        self.camera_height = camera_height_mm
        self.floor_depth = None
        self.history = deque(maxlen=history_size)
    
    def estimate_depth(self, frame):
        return self.model.infer_image(frame)
    
    def calibrate_floor(self, frame):
        depth = self.estimate_depth(frame)
        h, w = depth.shape
        center = depth[h//3:2*h//3, w//3:2*w//3]
        self.floor_depth = np.median(center)
        print(f"✅ Floor depth: {self.floor_depth:.4f}")
        print(f"   Camera height: {self.camera_height}mm")
        return self.floor_depth
    
    def get_object_height(self, depth_map, obj, scale):
        if self.floor_depth is None: return 0
        
        x, y, w, h = obj['bbox']
        region = depth_map[y:y+h, x:x+w]
        if region.size == 0: return 0
        
        # Multi-sample
        samples = [np.median(region)]
        qh, qw = h//4, w//4
        if qh > 0 and qw > 0:
            samples.extend([
                np.median(region[:qh, :qw]),
                np.median(region[:qh, -qw:]),
                np.median(region[-qh:, :qw]),
                np.median(region[-qh:, -qw:])
            ])
        
        obj_depth = np.median(samples)
        height = max(0, (obj_depth - self.floor_depth) * scale)
        
        # Temporal averaging
        self.history.append(height)
        return np.median(self.history)
    
    def calculate_z(self, height_mm):
        z = Z_FLOOR + (height_mm * 0.5)
        return max(Z_FLOOR, min(Z_SAFE, z))

print("✓ Depth estimator")

✓ Depth estimator


## 6️⃣ Initialize & Connect

In [16]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)
detector = PreciseSizeDetector(yolo_model, PIXELS_PER_MM)
grasp_selector = SmartGraspSelector(PIXELS_PER_MM)
depth_estimator = RobustDepthEstimator(depth_model, device=DEVICE, camera_height_mm=CAMERA_HEIGHT_MM)
print("✓ Components initialized")

✓ Components initialized


In [18]:
gripper.connect()

✅ Gripper on COM9


True

In [33]:
robot.connect(ROBOT_IP)

✅ Robot connected!


True

## 📷 Capture Background

In [30]:
print("📷 BACKGROUND | SPACE=Capture Q=Skip")
cap = cv2.VideoCapture(CAMERA_ID)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    cv2.putText(frame, "SPACE=Capture | Q=Skip", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    cv2.imshow('BG', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        depth_estimator.calibrate_floor(frame)
        break
    elif key == ord('q'):
        print("Skipped")
        break
cap.release()
cv2.destroyAllWindows()

📷 BACKGROUND | SPACE=Capture Q=Skip
✅ Floor depth: 3.0112
   Camera height: 630mm


---
# 🎯 MAIN PICK-AND-PLACE
---

In [51]:
robot.connect(ROBOT_IP)

✅ Robot connected!


True

In [45]:
robot.home()

🤖 HOME...


In [48]:
selected_object = None
selected_grasp = None
detected_objects = []
current_grasps = []
current_depth = None

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for g in current_grasps:
            gx, gy = g['center']
            if abs(x-gx) < 20 and abs(y-gy) < 20:
                selected_grasp = g
                print(f"\n🎯 Grasp: W={g['width_mm']:.1f}mm A={g['angle']:.1f}°")
                return
        for obj in detected_objects:
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    print(f"\n📦 Object: {len(current_grasps)} grasps")
                break

def draw_grasps(frame, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['angle']
        is_sel = (selected and g == selected)
        color = (0,0,255) if is_sel else ((0,255,0) if g['score']>=1.0 else (0,255,255))
        thick = 3 if is_sel else (2 if g['score']>=1.0 else 1)
        dx = int(30 * np.cos(np.radians(angle)))
        dy = int(30 * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx,cy-dy), (cx+dx,cy+dy), color, thick)
        cv2.circle(frame, (cx,cy), 5, color, -1)

def pick_with_grasp(obj, grasp):
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    robot_r = -grasp['angle']
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    height = depth_estimator.get_object_height(current_depth, obj, DEPTH_Z_SCALE) if current_depth is not None else 0
    z_grasp = depth_estimator.calculate_z(height)
    
    print(f"\n🤖 Pick: W={grip_w:.1f}mm R={robot_r:.1f}° Z={z_grasp:.1f}")
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(4)
    robot.move_to_and_wait(robot_x, robot_y, Z_APPROACH, robot_r, 3)
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 2)
    gripper.grip_object(grip_w-6) ### เพิ่ม -5 ให้แน่นขึ้น
    time.sleep(4)
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, robot_r, 2)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    gripper.release()
    time.sleep(4)
    robot.home()
    print("✅ Complete!")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v7')
cv2.setMouseCallback('Pick v7', mouse_callback)

frame_count = 0
print("="*50)
print("🎯 PICK v7")
print("Click=Select | SPACE=Execute | F=FrameGrid | O=ObjGrid | Q=Quit")
print("="*50)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    # Frame grid
    frame = detector.draw_frame_grid(frame)
    
    frame_count += 1
    if frame_count % 10 == 0:
        current_depth = depth_estimator.estimate_depth(frame)
    
    detected_objects = detector.detect(frame)
    
    for obj in detected_objects:
        x,y,w,h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0,0,255) if is_sel else (0,255,0)
        
        cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)
        if 'rect' in obj:
            box = cv2.boxPoints(obj['rect'])
            cv2.drawContours(frame, [np.int32(box)], 0, color, 1)
        
        # Object grid
        detector.draw_object_grid(frame, obj)
        
        rect_w, rect_h = obj['rect_size']
        w_mm = min(rect_w, rect_h) / PIXELS_PER_MM
        cv2.putText(frame, f"W:{w_mm:.0f}mm", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    if selected_object and current_grasps:
        draw_grasps(frame, current_grasps, selected_grasp)
    
    cv2.rectangle(frame, (0,0), (640,35), (30,30,30), -1)
    cv2.putText(frame, f"Obj:{len(detected_objects)} | Click | SPACE | F=FGrid O=OGrid | Q",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    if selected_grasp:
        cv2.putText(frame, f"[GRASP: W={selected_grasp['width_mm']:.1f}mm - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow('Pick v7', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('f'):
        SHOW_FRAME_GRID = not SHOW_FRAME_GRID
        print(f"Frame grid: {SHOW_FRAME_GRID}")
    elif key == ord('o'):
        SHOW_OBJECT_GRID = not SHOW_OBJECT_GRID
        print(f"Object grid: {SHOW_OBJECT_GRID}")
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_with_grasp(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()

🎯 PICK v7
Click=Select | SPACE=Execute | F=FrameGrid | O=ObjGrid | Q=Quit


In [ ]:
gripper.disconnect()
print("✅ Done")